In [42]:
# Import libraries
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_profiling
import datetime
sns.set(color_codes=True)
%matplotlib inline

import plotly.express as px

## Task 1. Import Data

In [2]:
# read data from csv file
contracts = pd.read_csv('contracts/Contracts.csv')
fuel_gda = pd.read_csv('fuelPrices/GDA_TETSTX.csv')
fuel_henry = pd.read_csv('fuelPrices/Henry Hub.csv')
plantParameters = pd.read_csv('plantParameters/Plant_Parameters.csv')
powerPrices_2016 = pd.read_csv('powerPrices/ERCOT_DA_Prices_2016.csv')
powerPrices_2017 = pd.read_csv('powerPrices/ERCOT_DA_Prices_2017.csv')
powerPrices_2018 = pd.read_csv('powerPrices/ERCOT_DA_Prices_2018.csv')
powerPrices_2019 = pd.read_csv('powerPrices/ERCOT_DA_Prices_2019.csv')

In [3]:
# fuel prices
fuelPrices = pd.concat([fuel_gda, fuel_henry], ignore_index=True)

In [4]:
fuelPrices

,Variable,Date,Price
0,GDA_TETSTX,2019-05-01,2.490
1,GDA_TETSTX,2019-04-30,2.420
2,GDA_TETSTX,2019-04-29,2.445
3,GDA_TETSTX,2019-04-28,2.445
4,GDA_TETSTX,2019-04-27,2.445
...,...,...,...
3677,Henry Hub,2020-09-26,1.895
3678,Henry Hub,2020-09-27,1.895
3679,Henry Hub,2020-09-28,1.895
3680,Henry Hub,2020-09-29,1.835


In [197]:
# power prices
powerPrices = pd.concat([powerPrices_2016, powerPrices_2017, powerPrices_2018, powerPrices_2019], ignore_index=True)

In [198]:
powerPrices

,Date,SettlementPoint,Price
0,2016-01-01 00:00:00,HB_BUSAVG,18.42
1,2016-01-01 00:00:00,HB_HOUSTON,18.42
2,2016-01-01 00:00:00,HB_HUBAVG,18.42
3,2016-01-01 00:00:00,HB_NORTH,18.43
4,2016-01-01 00:00:00,HB_SOUTH,18.38
...,...,...,...
497315,2019-12-31 23:00:00,LZ_LCRA,14.53
497316,2019-12-31 23:00:00,LZ_NORTH,14.48
497317,2019-12-31 23:00:00,LZ_RAYBN,14.48
497318,2019-12-31 23:00:00,LZ_SOUTH,14.70


## Task 2: Calculate basic descriptive statistics

In [7]:
powerPrices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497320 entries, 0 to 497319
Data columns (total 3 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Date             497320 non-null  object 
 1   SettlementPoint  497320 non-null  object 
 2   Price            497320 non-null  float64
dtypes: float64(1), object(2)
memory usage: 11.4+ MB


In [200]:
# convert to datetime
powerPrices['Date'] =  pd.to_datetime(powerPrices['Date'])

In [9]:
powerPrices.set_index(['Date'], inplace=True)

In [10]:
# group by SettlementPoint, Year, Month
a = powerPrices.groupby(['SettlementPoint',(powerPrices.index.year),(powerPrices.index.month)])

In [11]:
powerPrices_stats = a.agg(mean_lmp=pd.NamedAgg(column='Price', aggfunc='mean'),
      min_lmp=pd.NamedAgg(column='Price', aggfunc='min'),
      max_lmp=pd.NamedAgg(column='Price', aggfunc='max'),
      std_lmp=pd.NamedAgg(column='Price', aggfunc='std'))

In [25]:
powerPrices_stats.index.rename(['SettlementPoint', 'year', 'month'], inplace=True)

In [26]:
powerPrices_stats

mean_lmp  min_lmp  max_lmp     std_lmp
SettlementPoint year month                                          
HB_BUSAVG       2016 1       19.209301     5.12    54.69    5.485210
                     2       15.288247     1.79    50.98    5.777172
                     3       16.755626     1.22    64.31    7.368497
                     4       18.289347     1.18    93.09    9.506143
                     5       18.159583     3.03    73.61    8.017221
...                                ...      ...      ...         ...
LZ_WEST         2019 8      128.876465     9.12  3505.60  402.114146
                     9       83.471139     6.59  5015.25  380.453919
                     10      49.383360     0.79   873.07   62.396856
                     11      49.340847     3.60   286.91   31.876126
                     12      39.502594     7.32   195.71   27.975862

[681 rows x 4 columns]

## Task 3: Calculate volatility

In [13]:
powerPrices_pivot = powerPrices.pivot(columns='SettlementPoint', values='Price')

In [14]:
# log returns
Prices_log_returns = np.log(1+powerPrices_pivot.pct_change(1))

In [15]:
Prices_log_returns

SettlementPoint,HB_BUSAVG,HB_HOUSTON,HB_HUBAVG,HB_NORTH,HB_PAN,HB_SOUTH,HB_WEST,LZ_AEN,LZ_CPS,LZ_HOUSTON,LZ_LCRA,LZ_NORTH,LZ_RAYBN,LZ_SOUTH,LZ_WEST
Date,,,,,,,,,,,,,,,
2016-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-01 01:00:00,-0.131517,-0.131517,-0.132136,-0.130822,NaN,-0.133065,-0.131365,-0.131365,-0.130671,-0.132136,-0.131365,-0.130822,-0.130822,-0.135572,-0.131365
2016-01-01 02:00:00,-0.034008,-0.034008,-0.034030,-0.033326,NaN,-0.035424,-0.032687,-0.032687,-0.032646,-0.033389,-0.032687,-0.033326,-0.033326,-0.042368,-0.032687
2016-01-01 03:00:00,0.000640,0.000640,0.000641,0.000639,NaN,0.000000,0.000639,0.000639,0.001276,0.000640,0.000639,0.000639,0.000639,-0.001312,0.000639
2016-01-01 04:00:00,0.020280,0.020280,0.020920,0.020241,NaN,0.021027,0.020228,0.020228,0.019565,0.020280,0.020228,0.020241,0.020241,0.022721,0.020228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 19:00:00,-0.113805,-0.115898,-0.113862,-0.113977,-0.115046,-0.109291,-0.116825,-0.109472,-0.099072,-0.116555,-0.109163,-0.113158,-0.113633,-0.102782,-0.157093
2019-12-31 20:00:00,-0.074149,-0.072480,-0.074765,-0.074848,-0.094134,-0.069411,-0.082515,-0.072849,-0.061510,-0.073663,-0.070695,-0.075748,-0.075748,-0.066818,-0.060382
2019-12-31 21:00:00,-0.095138,-0.104981,-0.095253,-0.094735,-0.080728,-0.092576,-0.087384,-0.099695,-0.095848,-0.104983,-0.078530,-0.095770,-0.095770,-0.078873,-0.252222


In [16]:
# log returns
Prices_log_returns_month = Prices_log_returns.groupby([(Prices_log_returns.index.year),(Prices_log_returns.index.month)]).mean()

In [17]:
Prices_log_returns_month

SettlementPoint  HB_BUSAVG  HB_HOUSTON  HB_HUBAVG  HB_NORTH    HB_PAN  \
Date Date                                                               
2016 1           -0.000364   -0.000364  -0.000364 -0.000362       NaN   
     2           -0.000782   -0.000759  -0.000796 -0.000763       NaN   
     3            0.000774    0.000753   0.000787  0.000754       NaN   
     4           -0.000049   -0.000048  -0.000052 -0.000046       NaN   
     5            0.000150    0.000161   0.000158  0.000135       NaN   
     6            0.000483    0.000474   0.000479  0.000492       NaN   
     7           -0.000152   -0.000156  -0.000158 -0.000142       NaN   
     8            0.000093    0.000092   0.000098  0.000087       NaN   
     9           -0.000084   -0.000084  -0.000084 -0.000085       NaN   
     10          -0.000236   -0.000154  -0.000244 -0.000255       NaN   
     11           0.000225    0.000140   0.000234  0.000245       NaN   
     12           0.000318    0.000319   0.000317  0.000320       NaN   
2017 1           -0.000464   -0.000461  -0.000469 -0.000458       NaN   
     2           -0.000489   -0.000030  -0.000574 -0.000460       NaN   
     3            0.000338    0.000435   0.000425  0.000225       NaN   
     4            0.000506    0.000057   0.000516  0.000564       NaN   
     5           -0.000058   -0.000160  -0.000071 -0.000036       NaN   
     6            0.000054    0.000212   0.000078  0.000012       NaN   
     7           -0.000032   -0.000093  -0.000042 -0.000012       NaN   
     8           -0.000099   -0.000187  -0.000099 -0.000126       NaN   
     9           -0.000088   -0.000036  -0.000092 -0.000059       NaN   
     10          -0.000014    0.000109   0.000013 -0.000068       NaN   
     11          -0.000075   -0.000218  -0.000107 -0.000005       NaN   
     12           0.000192    0.000168   0.000187  0.000209       NaN   
2018 1           -0.000402    0.000002  -0.000389 -0.000463       NaN   
     2           -0.000134   -0.000550  -0.000173 -0.000034       NaN   
     3            0.000087    0.000229   0.000124  0.000002       NaN   
     4            0.000033   -0.000031  -0.000241  0.000287       NaN   
     5            0.000289    0.000243   0.000541  0.000080       NaN   
     6            0.000244    0.000215   0.000247  0.000259       NaN   
     7           -0.000087   -0.000112  -0.000088 -0.000089       NaN   
     8           -0.000114   -0.000086  -0.000106 -0.000139       NaN   
     9            0.000018    0.000016   0.000019  0.000018       NaN   
     10           0.000190    0.000202   0.000177  0.000215       NaN   
     11          -0.000203   -0.000234  -0.000522  0.000148       NaN   
     12          -0.000003   -0.000045   0.000311 -0.000344       NaN   
2019 1           -0.000072   -0.000032  -0.000084 -0.000062       NaN   
     2            0.000013   -0.000035   0.000024 -0.000003       NaN   
     3            0.000346    0.000374   0.000347  0.000363       NaN   
     4           -0.000492   -0.000472  -0.000600 -0.000348 -0.012479   
     5            0.000237    0.000225   0.000345  0.000091       NaN   
     6           -0.000283   -0.000310  -0.000277 -0.000271       NaN   
     7           -0.000030   -0.000002  -0.000038 -0.000045 -0.000124   
     8            0.000068    0.000070   0.000068  0.000066       NaN   
     9           -0.000268   -0.000056  -0.000317 -0.000286       NaN   
     10           0.000472    0.000272   0.000512  0.000506       NaN   
     11          -0.000419   -0.000340  -0.000475 -0.000416       NaN   
     12          -0.000087   -0.000171  -0.000028 -0.000105       NaN   

SettlementPoint  HB_SOUTH       HB_WEST    LZ_AEN    LZ_CPS  LZ_HOUSTON  \
Date Date                                                                 
2016 1          -0.000364 -3.630779e-04 -0.000363 -0.000363   -0.000364   
     2          -0.000790  6.772152e-04 -0.000737 -0.000745   -0.000759   
     3           0.000783  8.607201e-04  0.000732

In [18]:
Prices_log_returns_month = pd.melt(Prices_log_returns_month, value_vars=Prices_log_returns_month.columns, ignore_index=False)

In [19]:
Prices_log_returns_month.index.rename(['year', 'month'], inplace=True)

In [20]:
Prices_log_returns_month.reset_index(inplace=True)

In [23]:
Prices_log_returns_month.set_index(['SettlementPoint', 'year', 'month'], inplace=True)

In [27]:
Prices_log_returns_month

value
SettlementPoint year month          
HB_BUSAVG       2016 1     -0.000364
                     2     -0.000782
                     3      0.000774
                     4     -0.000049
                     5      0.000150
...                              ...
LZ_WEST         2019 8     -0.000010
                     9     -0.000488
                     10     0.001935
                     11     0.000226
                     12    -0.001834

[720 rows x 1 columns]

## Task 4: Write the results to file

In [29]:
# merge results
powerPrices_stats = powerPrices_stats.join(Prices_log_returns_month)

In [30]:
powerPrices_stats.columns = ['Mean', 'Min', 'Max', 'SD', 'Volatility']

In [32]:
# write to csv
powerPrices_stats.to_csv('MonthlyPowerPriceStatistics.csv')

## Task 5: Expand the contracts across relevant time periods

In [37]:
contracts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ContractName  4 non-null      object 
 1   DealType      4 non-null      object 
 2   StartDate     4 non-null      object 
 3   EndDate       4 non-null      object 
 4   Volume        4 non-null      int64  
 5   Granularity   4 non-null      object 
 6   StrikePrice   4 non-null      float64
 7   Premium       2 non-null      float64
 8   PriceName     4 non-null      object 
dtypes: float64(2), int64(1), object(6)
memory usage: 416.0+ bytes


In [38]:
# convert to datetime
contracts['StartDate'] =  pd.to_datetime(contracts['StartDate'])
contracts['EndDate'] =  pd.to_datetime(contracts['EndDate'])
contracts

,ContractName,DealType,StartDate,EndDate,Volume,Granularity,StrikePrice,Premium,PriceName
0,S1,Swap,2017-01-01,2017-03-31,20000,Daily,3.0,NaN,Henry Hub
1,S2,Swap,2017-01-01,2018-12-31,1000,Hourly,21.0,NaN,HB_NORTH
2,O1,European option,2017-06-01,2017-07-15,10000,Daily,2.9,0.1,GDA_TETSTX
3,O2,European option,2018-10-01,2019-09-30,1000,Hourly,31.5,3.0,HB_HOUSTON


In [39]:
contracts_daily = contracts[contracts['Granularity']=='Daily']
contracts_daily

,ContractName,DealType,StartDate,EndDate,Volume,Granularity,StrikePrice,Premium,PriceName
0,S1,Swap,2017-01-01,2017-03-31,20000,Daily,3.0,NaN,Henry Hub
2,O1,European option,2017-06-01,2017-07-15,10000,Daily,2.9,0.1,GDA_TETSTX


In [40]:
contracts_hourly = contracts[contracts['Granularity']=='Hourly']
contracts_hourly

,ContractName,DealType,StartDate,EndDate,Volume,Granularity,StrikePrice,Premium,PriceName
1,S2,Swap,2017-01-01,2018-12-31,1000,Hourly,21.0,NaN,HB_NORTH
3,O2,European option,2018-10-01,2019-09-30,1000,Hourly,31.5,3.0,HB_HOUSTON


In [77]:
delta = datetime.timedelta(days=1)
contracts_daily_expand = pd.DataFrame()
for index, row in contracts_daily.iterrows():
    start_date = row['StartDate']
    end_date = row['EndDate']
    while start_date <= end_date:
        new_row = pd.DataFrame([row['ContractName'], 
                               row['DealType'], 
                               start_date, 
                               row['Volume'], 
                               row['StrikePrice'], 
                               row['Premium'], 
                               row['PriceName']])
        contracts_daily_expand = pd.concat([contracts_daily_expand, new_row.T])
        # increment day
        print(start_date)
        start_date += delta
       

2017-01-01 00:00:00
2017-01-02 00:00:00
2017-01-03 00:00:00
2017-01-04 00:00:00
2017-01-05 00:00:00
2017-01-06 00:00:00
2017-01-07 00:00:00
2017-01-08 00:00:00
2017-01-09 00:00:00
2017-01-10 00:00:00
2017-01-11 00:00:00
2017-01-12 00:00:00
2017-01-13 00:00:00
2017-01-14 00:00:00
2017-01-15 00:00:00
2017-01-16 00:00:00
2017-01-17 00:00:00
2017-01-18 00:00:00
2017-01-19 00:00:00
2017-01-20 00:00:00
2017-01-21 00:00:00
2017-01-22 00:00:00
2017-01-23 00:00:00
2017-01-24 00:00:00
2017-01-25 00:00:00
2017-01-26 00:00:00
2017-01-27 00:00:00
2017-01-28 00:00:00
2017-01-29 00:00:00
2017-01-30 00:00:00
2017-01-31 00:00:00
2017-02-01 00:00:00
2017-02-02 00:00:00
2017-02-03 00:00:00
2017-02-04 00:00:00
2017-02-05 00:00:00
2017-02-06 00:00:00
2017-02-07 00:00:00
2017-02-08 00:00:00
2017-02-09 00:00:00
2017-02-10 00:00:00
2017-02-11 00:00:00
2017-02-12 00:00:00
2017-02-13 00:00:00
2017-02-14 00:00:00
2017-02-15 00:00:00
2017-02-16 00:00:00
2017-02-17 00:00:00
2017-02-18 00:00:00
2017-02-19 00:00:00


In [78]:
 
contracts_daily_expand.columns = ['ContractName', 'DealType', 'Date', 'Volume', 'StrikePrice', 'Premium', 'PriceName']
contracts_daily_expand

,ContractName,DealType,Date,Volume,StrikePrice,Premium,PriceName
0,S1,Swap,2017-01-01,20000,3.0,NaN,Henry Hub
0,S1,Swap,2017-01-02,20000,3.0,NaN,Henry Hub
0,S1,Swap,2017-01-03,20000,3.0,NaN,Henry Hub
0,S1,Swap,2017-01-04,20000,3.0,NaN,Henry Hub
0,S1,Swap,2017-01-05,20000,3.0,NaN,Henry Hub
...,...,...,...,...,...,...,...
0,O1,European option,2017-07-11,10000,2.9,0.1,GDA_TETSTX
0,O1,European option,2017-07-12,10000,2.9,0.1,GDA_TETSTX
0,O1,European option,2017-07-13,10000,2.9,0.1,GDA_TETSTX
0,O1,European option,2017-07-14,10000,2.9,0.1,GDA_TETSTX


In [79]:
delta = datetime.timedelta(hours=1)
contracts_hourly_expand = pd.DataFrame()
for index, row in contracts_hourly.iterrows():
    start_date = row['StartDate']
    end_date = row['EndDate']
    while start_date <= end_date:
        new_row = pd.DataFrame([row['ContractName'], 
                               row['DealType'], 
                               start_date, 
                               row['Volume'], 
                               row['StrikePrice'], 
                               row['Premium'], 
                               row['PriceName']])
        contracts_hourly_expand = pd.concat([contracts_hourly_expand, new_row.T])
        # increment day
        print(start_date)
        start_date += delta
   

2017-01-01 00:00:00
2017-01-01 01:00:00
2017-01-01 02:00:00
2017-01-01 03:00:00
2017-01-01 04:00:00
2017-01-01 05:00:00
2017-01-01 06:00:00
2017-01-01 07:00:00
2017-01-01 08:00:00
2017-01-01 09:00:00
2017-01-01 10:00:00
2017-01-01 11:00:00
2017-01-01 12:00:00
2017-01-01 13:00:00
2017-01-01 14:00:00
2017-01-01 15:00:00
2017-01-01 16:00:00
2017-01-01 17:00:00
2017-01-01 18:00:00
2017-01-01 19:00:00
2017-01-01 20:00:00
2017-01-01 21:00:00
2017-01-01 22:00:00
2017-01-01 23:00:00
2017-01-02 00:00:00
2017-01-02 01:00:00
2017-01-02 02:00:00
2017-01-02 03:00:00
2017-01-02 04:00:00
2017-01-02 05:00:00
2017-01-02 06:00:00
2017-01-02 07:00:00
2017-01-02 08:00:00
2017-01-02 09:00:00
2017-01-02 10:00:00
2017-01-02 11:00:00
2017-01-02 12:00:00
2017-01-02 13:00:00
2017-01-02 14:00:00
2017-01-02 15:00:00
2017-01-02 16:00:00
2017-01-02 17:00:00
2017-01-02 18:00:00
2017-01-02 19:00:00
2017-01-02 20:00:00
2017-01-02 21:00:00
2017-01-02 22:00:00
2017-01-02 23:00:00
2017-01-03 00:00:00
2017-01-03 01:00:00


2017-01-21 22:00:00
2017-01-21 23:00:00
2017-01-22 00:00:00
2017-01-22 01:00:00
2017-01-22 02:00:00
2017-01-22 03:00:00
2017-01-22 04:00:00
2017-01-22 05:00:00
2017-01-22 06:00:00
2017-01-22 07:00:00
2017-01-22 08:00:00
2017-01-22 09:00:00
2017-01-22 10:00:00
2017-01-22 11:00:00
2017-01-22 12:00:00
2017-01-22 13:00:00
2017-01-22 14:00:00
2017-01-22 15:00:00
2017-01-22 16:00:00
2017-01-22 17:00:00
2017-01-22 18:00:00
2017-01-22 19:00:00
2017-01-22 20:00:00
2017-01-22 21:00:00
2017-01-22 22:00:00
2017-01-22 23:00:00
2017-01-23 00:00:00
2017-01-23 01:00:00
2017-01-23 02:00:00
2017-01-23 03:00:00
2017-01-23 04:00:00
2017-01-23 05:00:00
2017-01-23 06:00:00
2017-01-23 07:00:00
2017-01-23 08:00:00
2017-01-23 09:00:00
2017-01-23 10:00:00
2017-01-23 11:00:00
2017-01-23 12:00:00
2017-01-23 13:00:00
2017-01-23 14:00:00
2017-01-23 15:00:00
2017-01-23 16:00:00
2017-01-23 17:00:00
2017-01-23 18:00:00
2017-01-23 19:00:00
2017-01-23 20:00:00
2017-01-23 21:00:00
2017-01-23 22:00:00
2017-01-23 23:00:00


2017-02-10 20:00:00
2017-02-10 21:00:00
2017-02-10 22:00:00
2017-02-10 23:00:00
2017-02-11 00:00:00
2017-02-11 01:00:00
2017-02-11 02:00:00
2017-02-11 03:00:00
2017-02-11 04:00:00
2017-02-11 05:00:00
2017-02-11 06:00:00
2017-02-11 07:00:00
2017-02-11 08:00:00
2017-02-11 09:00:00
2017-02-11 10:00:00
2017-02-11 11:00:00
2017-02-11 12:00:00
2017-02-11 13:00:00
2017-02-11 14:00:00
2017-02-11 15:00:00
2017-02-11 16:00:00
2017-02-11 17:00:00
2017-02-11 18:00:00
2017-02-11 19:00:00
2017-02-11 20:00:00
2017-02-11 21:00:00
2017-02-11 22:00:00
2017-02-11 23:00:00
2017-02-12 00:00:00
2017-02-12 01:00:00
2017-02-12 02:00:00
2017-02-12 03:00:00
2017-02-12 04:00:00
2017-02-12 05:00:00
2017-02-12 06:00:00
2017-02-12 07:00:00
2017-02-12 08:00:00
2017-02-12 09:00:00
2017-02-12 10:00:00
2017-02-12 11:00:00
2017-02-12 12:00:00
2017-02-12 13:00:00
2017-02-12 14:00:00
2017-02-12 15:00:00
2017-02-12 16:00:00
2017-02-12 17:00:00
2017-02-12 18:00:00
2017-02-12 19:00:00
2017-02-12 20:00:00
2017-02-12 21:00:00


2017-03-01 22:00:00
2017-03-01 23:00:00
2017-03-02 00:00:00
2017-03-02 01:00:00
2017-03-02 02:00:00
2017-03-02 03:00:00
2017-03-02 04:00:00
2017-03-02 05:00:00
2017-03-02 06:00:00
2017-03-02 07:00:00
2017-03-02 08:00:00
2017-03-02 09:00:00
2017-03-02 10:00:00
2017-03-02 11:00:00
2017-03-02 12:00:00
2017-03-02 13:00:00
2017-03-02 14:00:00
2017-03-02 15:00:00
2017-03-02 16:00:00
2017-03-02 17:00:00
2017-03-02 18:00:00
2017-03-02 19:00:00
2017-03-02 20:00:00
2017-03-02 21:00:00
2017-03-02 22:00:00
2017-03-02 23:00:00
2017-03-03 00:00:00
2017-03-03 01:00:00
2017-03-03 02:00:00
2017-03-03 03:00:00
2017-03-03 04:00:00
2017-03-03 05:00:00
2017-03-03 06:00:00
2017-03-03 07:00:00
2017-03-03 08:00:00
2017-03-03 09:00:00
2017-03-03 10:00:00
2017-03-03 11:00:00
2017-03-03 12:00:00
2017-03-03 13:00:00
2017-03-03 14:00:00
2017-03-03 15:00:00
2017-03-03 16:00:00
2017-03-03 17:00:00
2017-03-03 18:00:00
2017-03-03 19:00:00
2017-03-03 20:00:00
2017-03-03 21:00:00
2017-03-03 22:00:00
2017-03-03 23:00:00


2017-03-20 03:00:00
2017-03-20 04:00:00
2017-03-20 05:00:00
2017-03-20 06:00:00
2017-03-20 07:00:00
2017-03-20 08:00:00
2017-03-20 09:00:00
2017-03-20 10:00:00
2017-03-20 11:00:00
2017-03-20 12:00:00
2017-03-20 13:00:00
2017-03-20 14:00:00
2017-03-20 15:00:00
2017-03-20 16:00:00
2017-03-20 17:00:00
2017-03-20 18:00:00
2017-03-20 19:00:00
2017-03-20 20:00:00
2017-03-20 21:00:00
2017-03-20 22:00:00
2017-03-20 23:00:00
2017-03-21 00:00:00
2017-03-21 01:00:00
2017-03-21 02:00:00
2017-03-21 03:00:00
2017-03-21 04:00:00
2017-03-21 05:00:00
2017-03-21 06:00:00
2017-03-21 07:00:00
2017-03-21 08:00:00
2017-03-21 09:00:00
2017-03-21 10:00:00
2017-03-21 11:00:00
2017-03-21 12:00:00
2017-03-21 13:00:00
2017-03-21 14:00:00
2017-03-21 15:00:00
2017-03-21 16:00:00
2017-03-21 17:00:00
2017-03-21 18:00:00
2017-03-21 19:00:00
2017-03-21 20:00:00
2017-03-21 21:00:00
2017-03-21 22:00:00
2017-03-21 23:00:00
2017-03-22 00:00:00
2017-03-22 01:00:00
2017-03-22 02:00:00
2017-03-22 03:00:00
2017-03-22 04:00:00


...

2019-07-21 00:00:00
2019-07-21 01:00:00
2019-07-21 02:00:00
2019-07-21 03:00:00
2019-07-21 04:00:00
2019-07-21 05:00:00
2019-07-21 06:00:00
2019-07-21 07:00:00
2019-07-21 08:00:00
2019-07-21 09:00:00
2019-07-21 10:00:00
2019-07-21 11:00:00
2019-07-21 12:00:00
2019-07-21 13:00:00
2019-07-21 14:00:00
2019-07-21 15:00:00
2019-07-21 16:00:00
2019-07-21 17:00:00
2019-07-21 18:00:00
2019-07-21 19:00:00
2019-07-21 20:00:00
2019-07-21 21:00:00
2019-07-21 22:00:00
2019-07-21 23:00:00
2019-07-22 00:00:00
2019-07-22 01:00:00
2019-07-22 02:00:00
2019-07-22 03:00:00
2019-07-22 04:00:00
2019-07-22 05:00:00
2019-07-22 06:00:00
2019-07-22 07:00:00
2019-07-22 08:00:00
2019-07-22 09:00:00
2019-07-22 10:00:00
2019-07-22 11:00:00
2019-07-22 12:00:00
2019-07-22 13:00:00
2019-07-22 14:00:00
2019-07-22 15:00:00
2019-07-22 16:00:00
2019-07-22 17:00:00
2019-07-22 18:00:00
2019-07-22 19:00:00
2019-07-22 20:00:00
2019-07-22 21:00:00
2019-07-22 22:00:00
2019-07-22 23:00:00
2019-07-23 00:00:00
2019-07-23 01:00:00


2019-08-08 03:00:00
2019-08-08 04:00:00
2019-08-08 05:00:00
2019-08-08 06:00:00
2019-08-08 07:00:00
2019-08-08 08:00:00
2019-08-08 09:00:00
2019-08-08 10:00:00
2019-08-08 11:00:00
2019-08-08 12:00:00
2019-08-08 13:00:00
2019-08-08 14:00:00
2019-08-08 15:00:00
2019-08-08 16:00:00
2019-08-08 17:00:00
2019-08-08 18:00:00
2019-08-08 19:00:00
2019-08-08 20:00:00
2019-08-08 21:00:00
2019-08-08 22:00:00
2019-08-08 23:00:00
2019-08-09 00:00:00
2019-08-09 01:00:00
2019-08-09 02:00:00
2019-08-09 03:00:00
2019-08-09 04:00:00
2019-08-09 05:00:00
2019-08-09 06:00:00
2019-08-09 07:00:00
2019-08-09 08:00:00
2019-08-09 09:00:00
2019-08-09 10:00:00
2019-08-09 11:00:00
2019-08-09 12:00:00
2019-08-09 13:00:00
2019-08-09 14:00:00
2019-08-09 15:00:00
2019-08-09 16:00:00
2019-08-09 17:00:00
2019-08-09 18:00:00
2019-08-09 19:00:00
2019-08-09 20:00:00
2019-08-09 21:00:00
2019-08-09 22:00:00
2019-08-09 23:00:00
2019-08-10 00:00:00
2019-08-10 01:00:00
2019-08-10 02:00:00
2019-08-10 03:00:00
2019-08-10 04:00:00


2019-08-26 09:00:00
2019-08-26 10:00:00
2019-08-26 11:00:00
2019-08-26 12:00:00
2019-08-26 13:00:00
2019-08-26 14:00:00
2019-08-26 15:00:00
2019-08-26 16:00:00
2019-08-26 17:00:00
2019-08-26 18:00:00
2019-08-26 19:00:00
2019-08-26 20:00:00
2019-08-26 21:00:00
2019-08-26 22:00:00
2019-08-26 23:00:00
2019-08-27 00:00:00
2019-08-27 01:00:00
2019-08-27 02:00:00
2019-08-27 03:00:00
2019-08-27 04:00:00
2019-08-27 05:00:00
2019-08-27 06:00:00
2019-08-27 07:00:00
2019-08-27 08:00:00
2019-08-27 09:00:00
2019-08-27 10:00:00
2019-08-27 11:00:00
2019-08-27 12:00:00
2019-08-27 13:00:00
2019-08-27 14:00:00
2019-08-27 15:00:00
2019-08-27 16:00:00
2019-08-27 17:00:00
2019-08-27 18:00:00
2019-08-27 19:00:00
2019-08-27 20:00:00
2019-08-27 21:00:00
2019-08-27 22:00:00
2019-08-27 23:00:00
2019-08-28 00:00:00
2019-08-28 01:00:00
2019-08-28 02:00:00
2019-08-28 03:00:00
2019-08-28 04:00:00
2019-08-28 05:00:00
2019-08-28 06:00:00
2019-08-28 07:00:00
2019-08-28 08:00:00
2019-08-28 09:00:00
2019-08-28 10:00:00


2019-09-12 12:00:00
2019-09-12 13:00:00
2019-09-12 14:00:00
2019-09-12 15:00:00
2019-09-12 16:00:00
2019-09-12 17:00:00
2019-09-12 18:00:00
2019-09-12 19:00:00
2019-09-12 20:00:00
2019-09-12 21:00:00
2019-09-12 22:00:00
2019-09-12 23:00:00
2019-09-13 00:00:00
2019-09-13 01:00:00
2019-09-13 02:00:00
2019-09-13 03:00:00
2019-09-13 04:00:00
2019-09-13 05:00:00
2019-09-13 06:00:00
2019-09-13 07:00:00
2019-09-13 08:00:00
2019-09-13 09:00:00
2019-09-13 10:00:00
2019-09-13 11:00:00
2019-09-13 12:00:00
2019-09-13 13:00:00
2019-09-13 14:00:00
2019-09-13 15:00:00
2019-09-13 16:00:00
2019-09-13 17:00:00
2019-09-13 18:00:00
2019-09-13 19:00:00
2019-09-13 20:00:00
2019-09-13 21:00:00
2019-09-13 22:00:00
2019-09-13 23:00:00
2019-09-14 00:00:00
2019-09-14 01:00:00
2019-09-14 02:00:00
2019-09-14 03:00:00
2019-09-14 04:00:00
2019-09-14 05:00:00
2019-09-14 06:00:00
2019-09-14 07:00:00
2019-09-14 08:00:00
2019-09-14 09:00:00
2019-09-14 10:00:00
2019-09-14 11:00:00
2019-09-14 12:00:00
2019-09-14 13:00:00


2019-09-29 14:00:00
2019-09-29 15:00:00
2019-09-29 16:00:00
2019-09-29 17:00:00
2019-09-29 18:00:00
2019-09-29 19:00:00
2019-09-29 20:00:00
2019-09-29 21:00:00
2019-09-29 22:00:00
2019-09-29 23:00:00
2019-09-30 00:00:00


In [80]:
     
contracts_hourly_expand.columns = ['ContractName', 'DealType', 'Date', 'Volume', 'StrikePrice', 'Premium', 'PriceName']
contracts_hourly_expand

,ContractName,DealType,Date,Volume,StrikePrice,Premium,PriceName
0,S2,Swap,2017-01-01 00:00:00,1000,21.0,NaN,HB_NORTH
0,S2,Swap,2017-01-01 01:00:00,1000,21.0,NaN,HB_NORTH
0,S2,Swap,2017-01-01 02:00:00,1000,21.0,NaN,HB_NORTH
0,S2,Swap,2017-01-01 03:00:00,1000,21.0,NaN,HB_NORTH
0,S2,Swap,2017-01-01 04:00:00,1000,21.0,NaN,HB_NORTH
...,...,...,...,...,...,...,...
0,O2,European option,2019-09-29 20:00:00,1000,31.5,3.0,HB_HOUSTON
0,O2,European option,2019-09-29 21:00:00,1000,31.5,3.0,HB_HOUSTON
0,O2,European option,2019-09-29 22:00:00,1000,31.5,3.0,HB_HOUSTON
0,O2,European option,2019-09-29 23:00:00,1000,31.5,3.0,HB_HOUSTON


In [81]:
contracts_hourly_expand

,ContractName,DealType,Date,Volume,StrikePrice,Premium,PriceName
0,S2,Swap,2017-01-01 00:00:00,1000,21.0,NaN,HB_NORTH
0,S2,Swap,2017-01-01 01:00:00,1000,21.0,NaN,HB_NORTH
0,S2,Swap,2017-01-01 02:00:00,1000,21.0,NaN,HB_NORTH
0,S2,Swap,2017-01-01 03:00:00,1000,21.0,NaN,HB_NORTH
0,S2,Swap,2017-01-01 04:00:00,1000,21.0,NaN,HB_NORTH
...,...,...,...,...,...,...,...
0,O2,European option,2019-09-29 20:00:00,1000,31.5,3.0,HB_HOUSTON
0,O2,European option,2019-09-29 21:00:00,1000,31.5,3.0,HB_HOUSTON
0,O2,European option,2019-09-29 22:00:00,1000,31.5,3.0,HB_HOUSTON
0,O2,European option,2019-09-29 23:00:00,1000,31.5,3.0,HB_HOUSTON


## Task 6: Join relevant prices

In [86]:
fuelPrices['Date'] =  pd.to_datetime(fuelPrices['Date'])

In [99]:
fuelPrices_merge = fuelPrices.merge(contracts_daily_expand, left_on=['Variable', 'Date'], right_on=['PriceName', 'Date'])

In [100]:
fuelPrices_merge

,Variable,Date,Price,ContractName,DealType,Volume,StrikePrice,Premium,PriceName
0,GDA_TETSTX,2017-07-15,2.935,O1,European option,10000,2.9,0.1,GDA_TETSTX
1,GDA_TETSTX,2017-07-14,2.865,O1,European option,10000,2.9,0.1,GDA_TETSTX
2,GDA_TETSTX,2017-07-13,2.915,O1,European option,10000,2.9,0.1,GDA_TETSTX
3,GDA_TETSTX,2017-07-12,2.910,O1,European option,10000,2.9,0.1,GDA_TETSTX
4,GDA_TETSTX,2017-07-11,2.910,O1,European option,10000,2.9,0.1,GDA_TETSTX
...,...,...,...,...,...,...,...,...,...
130,Henry Hub,2017-03-27,2.925,S1,Swap,20000,3.0,NaN,Henry Hub
131,Henry Hub,2017-03-28,2.955,S1,Swap,20000,3.0,NaN,Henry Hub
132,Henry Hub,2017-03-29,2.935,S1,Swap,20000,3.0,NaN,Henry Hub
133,Henry Hub,2017-03-30,3.035,S1,Swap,20000,3.0,NaN,Henry Hub


In [92]:
powerPrices.reset_index(inplace=True)

In [95]:
powerPrices_merge = powerPrices.merge(contracts_hourly_expand, left_on=['SettlementPoint', 'Date'], right_on=['PriceName', 'Date'])

In [96]:
powerPrices_merge

,Date,SettlementPoint,Price,ContractName,DealType,Volume,StrikePrice,Premium,PriceName
0,2017-01-01 00:00:00,HB_NORTH,23.23,S2,Swap,1000,21.0,NaN,HB_NORTH
1,2017-01-01 01:00:00,HB_NORTH,21.82,S2,Swap,1000,21.0,NaN,HB_NORTH
2,2017-01-01 02:00:00,HB_NORTH,20.16,S2,Swap,1000,21.0,NaN,HB_NORTH
3,2017-01-01 03:00:00,HB_NORTH,20.00,S2,Swap,1000,21.0,NaN,HB_NORTH
4,2017-01-01 04:00:00,HB_NORTH,20.24,S2,Swap,1000,21.0,NaN,HB_NORTH
...,...,...,...,...,...,...,...,...,...
26226,2019-09-29 20:00:00,HB_HOUSTON,23.71,O2,European option,1000,31.5,3.0,HB_HOUSTON
26227,2019-09-29 21:00:00,HB_HOUSTON,20.95,O2,European option,1000,31.5,3.0,HB_HOUSTON
26228,2019-09-29 22:00:00,HB_HOUSTON,18.56,O2,European option,1000,31.5,3.0,HB_HOUSTON
26229,2019-09-29 23:00:00,HB_HOUSTON,17.89,O2,European option,1000,31.5,3.0,HB_HOUSTON


## Task 7: Calculate payoffs

In [117]:
# Swaps: (AssetPrice – StrikePrice) x Volume
ix_swap = fuelPrices_merge['DealType']=='Swap'
fuelPrices_merge.loc[ix_swap, 'PayOff'] = (fuelPrices_merge.loc[ix_swap, 'Price']-fuelPrices_merge.loc[ix_swap, 'StrikePrice'])*fuelPrices_merge.loc[ix_swap, 'Volume']

In [118]:
# Options: [MAX(AssetPrice – StrikePrice, 0) - Premium] x Volume
ix_option = fuelPrices_merge['DealType']=='European option'
ix_pos = fuelPrices_merge['Price']>fuelPrices_merge['StrikePrice']
fuelPrices_merge.loc[ix_option & ix_pos, 'PayOff'] = (fuelPrices_merge.loc[ix_option & ix_pos, 'Price']-fuelPrices_merge.loc[ix_option & ix_pos, 'StrikePrice']-fuelPrices_merge.loc[ix_option & ix_pos, 'Premium'])*fuelPrices_merge.loc[ix_option & ix_pos, 'Volume']

ix_neg = fuelPrices_merge['Price']<fuelPrices_merge['StrikePrice']
fuelPrices_merge.loc[ix_option & ix_neg, 'PayOff'] = -fuelPrices_merge.loc[ix_option & ix_neg, 'Premium']*fuelPrices_merge.loc[ix_option & ix_neg, 'Volume']

In [119]:
fuelPrices_merge

,Variable,Date,Price,ContractName,DealType,Volume,StrikePrice,Premium,PriceName,PayOff
0,GDA_TETSTX,2017-07-15,2.935,O1,European option,10000,2.9,0.1,GDA_TETSTX,-650.0
1,GDA_TETSTX,2017-07-14,2.865,O1,European option,10000,2.9,0.1,GDA_TETSTX,-1000.0
2,GDA_TETSTX,2017-07-13,2.915,O1,European option,10000,2.9,0.1,GDA_TETSTX,-850.0
3,GDA_TETSTX,2017-07-12,2.910,O1,European option,10000,2.9,0.1,GDA_TETSTX,-900.0
4,GDA_TETSTX,2017-07-11,2.910,O1,European option,10000,2.9,0.1,GDA_TETSTX,-900.0
...,...,...,...,...,...,...,...,...,...,...
130,Henry Hub,2017-03-27,2.925,S1,Swap,20000,3.0,NaN,Henry Hub,-1500.0
131,Henry Hub,2017-03-28,2.955,S1,Swap,20000,3.0,NaN,Henry Hub,-900.0
132,Henry Hub,2017-03-29,2.935,S1,Swap,20000,3.0,NaN,Henry Hub,-1300.0
133,Henry Hub,2017-03-30,3.035,S1,Swap,20000,3.0,NaN,Henry Hub,700.0


In [120]:
# Swaps: (AssetPrice – StrikePrice) x Volume
ix_swap = powerPrices_merge['DealType']=='Swap'
powerPrices_merge.loc[ix_swap, 'PayOff'] = (powerPrices_merge.loc[ix_swap, 'Price']-powerPrices_merge.loc[ix_swap, 'StrikePrice'])*powerPrices_merge.loc[ix_swap, 'Volume']

In [121]:
# Options: [MAX(AssetPrice – StrikePrice, 0) - Premium] x Volume
ix_option = powerPrices_merge['DealType']=='European option'
ix_pos = powerPrices_merge['Price']>powerPrices_merge['StrikePrice']
powerPrices_merge.loc[ix_option & ix_pos, 'PayOff'] = (powerPrices_merge.loc[ix_option & ix_pos, 'Price']-powerPrices_merge.loc[ix_option & ix_pos, 'StrikePrice']-powerPrices_merge.loc[ix_option & ix_pos, 'Premium'])*powerPrices_merge.loc[ix_option & ix_pos, 'Volume']

ix_neg = powerPrices_merge['Price']<powerPrices_merge['StrikePrice']
powerPrices_merge.loc[ix_option & ix_neg, 'PayOff'] = -powerPrices_merge.loc[ix_option & ix_neg, 'Premium']*powerPrices_merge.loc[ix_option & ix_neg, 'Volume']

In [122]:
powerPrices_merge

,Date,SettlementPoint,Price,ContractName,DealType,Volume,StrikePrice,Premium,PriceName,PayOff
0,2017-01-01 00:00:00,HB_NORTH,23.23,S2,Swap,1000,21.0,NaN,HB_NORTH,2230.0
1,2017-01-01 01:00:00,HB_NORTH,21.82,S2,Swap,1000,21.0,NaN,HB_NORTH,820.0
2,2017-01-01 02:00:00,HB_NORTH,20.16,S2,Swap,1000,21.0,NaN,HB_NORTH,-840.0
3,2017-01-01 03:00:00,HB_NORTH,20.00,S2,Swap,1000,21.0,NaN,HB_NORTH,-1000.0
4,2017-01-01 04:00:00,HB_NORTH,20.24,S2,Swap,1000,21.0,NaN,HB_NORTH,-760.0
...,...,...,...,...,...,...,...,...,...,...
26226,2019-09-29 20:00:00,HB_HOUSTON,23.71,O2,European option,1000,31.5,3.0,HB_HOUSTON,-3000.0
26227,2019-09-29 21:00:00,HB_HOUSTON,20.95,O2,European option,1000,31.5,3.0,HB_HOUSTON,-3000.0
26228,2019-09-29 22:00:00,HB_HOUSTON,18.56,O2,European option,1000,31.5,3.0,HB_HOUSTON,-3000.0
26229,2019-09-29 23:00:00,HB_HOUSTON,17.89,O2,European option,1000,31.5,3.0,HB_HOUSTON,-3000.0


## Task 8: Calculate aggregated payoffs

In [123]:
fuelPrices_merge.set_index(['Date'], inplace=True)

In [124]:
# group by ContractName, Year, Month
b = fuelPrices_merge.groupby(['ContractName',(fuelPrices_merge.index.year),(fuelPrices_merge.index.month)])

In [125]:
fuelPrices_merge_total = b.agg(sum_payoff=pd.NamedAgg(column='PayOff', aggfunc='sum'))

In [126]:
fuelPrices_merge_total

sum_payoff
ContractName Date Date            
O1           2017 6       -14100.0
                  7       -14300.0
S1           2017 1       198300.0
                  2       -97700.0
                  3       -97700.0

In [127]:
powerPrices_merge.set_index(['Date'], inplace=True)

In [128]:
# group by ContractName, Year, Month
b = powerPrices_merge.groupby(['ContractName',(powerPrices_merge.index.year),(powerPrices_merge.index.month)])

In [129]:
powerPrices_merge_total = b.agg(sum_payoff=pd.NamedAgg(column='PayOff', aggfunc='sum'))

In [130]:
powerPrices_merge_total

sum_payoff
ContractName Date Date            
O2           2018 10      513430.0
                  11      598010.0
                  12       43140.0
             2019 1     -1320540.0
                  2     -1680820.0
                  3       678800.0
                  4      -727570.0
                  5     -1097720.0
                  6       773630.0
                  7      3355940.0
                  8     75340320.0
                  9     39174380.0
S2           2017 1      1481940.0
                  2     -1042810.0
                  3       169240.0
                  4       295600.0
                  5      1800420.0
                  6      3521210.0
                  7      7182810.0
                  8      4339550.0
                  9      1689200.0
                  10      301990.0
                  11      429270.0
                  12      -34970.0
             2018 1     12339910.0
                  2       515440.0
                  3      1194750.0
                  4      1816500.0
                  5     10643770.0
                  6      6632560.0
                  7     43128530.0
                  8      7888670.0
                  9      4535740.0
                  10     6721420.0
                  11     7369120.0
                  12     6966000.0

## Task 9: Write the results to file

In [140]:
contract_merge = pd.concat([fuelPrices_merge_total, powerPrices_merge_total])

In [141]:
contract_merge.index.rename(['ContractName', 'year', 'month'], inplace=True)

In [142]:
contract_merge.columns = ['TotalPayoff']

In [144]:
contract_merge.to_csv('MonthlyContractPayoffs.csv')

## Task 10: Calculate the hourly running cost of each power plant

In [145]:
plantParameters

,PlantName,Year,Month,PowerPriceName,FuelPriceName,Capacity,HeatRate,VOM,FuelTransportationCost,FixedStartCost
0,Alpha,2017,1,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000
1,Alpha,2017,2,HB_HOUSTON,Henry Hub,98,9.4,3,0.10,10000
2,Alpha,2017,3,HB_HOUSTON,Henry Hub,96,9.6,3,0.10,10000
3,Alpha,2017,4,HB_HOUSTON,Henry Hub,94,9.8,3,0.10,10000
4,Alpha,2017,5,HB_HOUSTON,Henry Hub,92,10.0,3,0.10,10000
...,...,...,...,...,...,...,...,...,...,...
43,Bravo,2018,8,HB_SOUTH,GDA_TETSTX,230,7.8,2,0.05,50000
44,Bravo,2018,9,HB_SOUTH,GDA_TETSTX,235,7.8,2,0.05,50000
45,Bravo,2018,10,HB_SOUTH,GDA_TETSTX,240,7.7,2,0.05,50000
46,Bravo,2018,11,HB_SOUTH,GDA_TETSTX,245,7.7,2,0.05,50000


In [148]:
fuelPrices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3682 entries, 0 to 3681
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Variable  3682 non-null   object        
 1   Date      3682 non-null   datetime64[ns]
 2   Price     3682 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 86.4+ KB


In [156]:
fuelPrices['Year'] = fuelPrices['Date'].dt.year
fuelPrices['Month'] = fuelPrices['Date'].dt.month

In [163]:
fuelPrices.columns = ['FuelPriceName', 'Date', 'Price', 'Year', 'Month']

In [167]:
plantParameters_merge = plantParameters.merge(fuelPrices, how='cross')

In [168]:
ix_fuel = plantParameters_merge['FuelPriceName_x']==plantParameters_merge['FuelPriceName_y']
ix_year = plantParameters_merge['Year_x']==plantParameters_merge['Year_y']
ix_month = plantParameters_merge['Month_x']==plantParameters_merge['Month_y']

In [170]:
plantParameters_merge = plantParameters_merge.loc[ix_fuel & ix_year & ix_month]

In [172]:
plantParameters_merge.reset_index(inplace=True)

In [174]:
# ((FuelPrice + FuelTransportationCost) x HeatRate) + VOM
plantParameters_merge['RunningCost'] = ((plantParameters_merge['Price'] + plantParameters_merge['FuelTransportationCost'])*plantParameters_merge['HeatRate']) + plantParameters_merge['VOM']

In [175]:
plantParameters_merge

,index,PlantName,Year_x,Month_x,PowerPriceName,FuelPriceName_x,Capacity,HeatRate,VOM,FuelTransportationCost,FixedStartCost,FuelPriceName_y,Date,Price,Year_y,Month_y,RunningCost
0,2313,Alpha,2017,1,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,Henry Hub,2017-01-01,3.650,2017,1,37.500
1,2314,Alpha,2017,1,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,Henry Hub,2017-01-02,3.650,2017,1,37.500
2,2315,Alpha,2017,1,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,Henry Hub,2017-01-03,3.650,2017,1,37.500
3,2316,Alpha,2017,1,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,Henry Hub,2017-01-04,3.390,2017,1,35.108
4,2317,Alpha,2017,1,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,Henry Hub,2017-01-05,3.370,2017,1,34.924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,173201,Bravo,2018,12,HB_SOUTH,GDA_TETSTX,250,7.6,2,0.05,50000,GDA_TETSTX,2018-12-05,4.435,2018,12,36.086
1456,173202,Bravo,2018,12,HB_SOUTH,GDA_TETSTX,250,7.6,2,0.05,50000,GDA_TETSTX,2018-12-04,4.370,2018,12,35.592
1457,173203,Bravo,2018,12,HB_SOUTH,GDA_TETSTX,250,7.6,2,0.05,50000,GDA_TETSTX,2018-12-03,4.375,2018,12,35.630
1458,173204,Bravo,2018,12,HB_SOUTH,GDA_TETSTX,250,7.6,2,0.05,50000,GDA_TETSTX,2018-12-02,4.375,2018,12,35.630


## Task 11: Join the hourly power prices

In [214]:
powerPrices['Date1'] = powerPrices['Date'].dt.date
powerPrices['Year'] = powerPrices['Date'].dt.year

In [215]:
powerPrices['Date1'] =  pd.to_datetime(powerPrices['Date1'])

In [216]:
plantParameters['PowerPriceName'].unique()

array(['HB_HOUSTON', 'HB_SOUTH'], dtype=object)

In [217]:
ix_node = (powerPrices['SettlementPoint']=='HB_HOUSTON') | (powerPrices['SettlementPoint']=='HB_SOUTH')
ix_year = (powerPrices['Year']==2017) | (powerPrices['Year']==2018)
powerPrices_sub = powerPrices.loc[ix_node & ix_year]

In [218]:
powerPrices_sub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35036 entries, 122963 to 368204
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             35036 non-null  datetime64[ns]
 1   SettlementPoint  35036 non-null  object        
 2   Price            35036 non-null  float64       
 3   Date1            35036 non-null  datetime64[ns]
 4   Year             35036 non-null  int64         
dtypes: datetime64[ns](2), float64(1), int64(1), object(1)
memory usage: 1.6+ MB


In [219]:
powerPrices_sub

,Date,SettlementPoint,Price,Date1,Year
122963,2017-01-01 00:00:00,HB_HOUSTON,23.23,2017-01-01,2017
122966,2017-01-01 00:00:00,HB_SOUTH,23.23,2017-01-01,2017
122977,2017-01-01 01:00:00,HB_HOUSTON,21.82,2017-01-01,2017
122980,2017-01-01 01:00:00,HB_SOUTH,21.82,2017-01-01,2017
122991,2017-01-01 02:00:00,HB_HOUSTON,20.16,2017-01-01,2017
...,...,...,...,...,...
368176,2018-12-31 21:00:00,HB_SOUTH,24.09,2018-12-31,2018
368187,2018-12-31 22:00:00,HB_HOUSTON,22.03,2018-12-31,2018
368190,2018-12-31 22:00:00,HB_SOUTH,22.67,2018-12-31,2018
368201,2018-12-31 23:00:00,HB_HOUSTON,20.00,2018-12-31,2018


In [220]:
plantParameters_merge_prices = powerPrices_sub.merge(plantParameters_merge,
                                                     how='left',
                                                     left_on=['SettlementPoint', 'Date1'],
                                                     right_on=['PowerPriceName', 'Date'])

In [221]:
plantParameters_merge_prices

,Date_x,SettlementPoint,Price_x,Date1,Year,index,PlantName,Year_x,Month_x,PowerPriceName,...,HeatRate,VOM,FuelTransportationCost,FixedStartCost,FuelPriceName_y,Date_y,Price_y,Year_y,Month_y,RunningCost
0,2017-01-01 00:00:00,HB_HOUSTON,23.23,2017-01-01,2017,2313,Alpha,2017,1,HB_HOUSTON,...,9.2,3,0.10,10000,Henry Hub,2017-01-01,3.650,2017,1,37.500
1,2017-01-01 00:00:00,HB_SOUTH,23.23,2017-01-01,2017,89218,Bravo,2017,1,HB_SOUTH,...,7.6,2,0.05,50000,GDA_TETSTX,2017-01-01,3.505,2017,1,29.018
2,2017-01-01 01:00:00,HB_HOUSTON,21.82,2017-01-01,2017,2313,Alpha,2017,1,HB_HOUSTON,...,9.2,3,0.10,10000,Henry Hub,2017-01-01,3.650,2017,1,37.500
3,2017-01-01 01:00:00,HB_SOUTH,21.82,2017-01-01,2017,89218,Bravo,2017,1,HB_SOUTH,...,7.6,2,0.05,50000,GDA_TETSTX,2017-01-01,3.505,2017,1,29.018
4,2017-01-01 02:00:00,HB_HOUSTON,20.16,2017-01-01,2017,2313,Alpha,2017,1,HB_HOUSTON,...,9.2,3,0.10,10000,Henry Hub,2017-01-01,3.650,2017,1,37.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35031,2018-12-31 21:00:00,HB_SOUTH,24.09,2018-12-31,2018,173175,Bravo,2018,12,HB_SOUTH,...,7.6,2,0.05,50000,GDA_TETSTX,2018-12-31,3.080,2018,12,25.788
35032,2018-12-31 22:00:00,HB_HOUSTON,22.03,2018-12-31,2018,87728,Alpha,2018,12,HB_HOUSTON,...,9.2,3,0.10,10000,Henry Hub,2018-12-31,3.065,2018,12,32.118
35033,2018-12-31 22:00:00,HB_SOUTH,22.67,2018-12-31,2018,173175,Bravo,2018,12,HB_SOUTH,...,7.6,2,0.05,50000,GDA_TETSTX,2018-12-31,3.080,2018,12,25.788
35034,2018-12-31 23:00:00,HB_HOUSTON,20.00,2018-12-31,2018,87728,Alpha,2018,12,HB_HOUSTON,...,9.2,3,0.10,10000,Henry Hub,2018-12-31,3.065,2018,12,32.118


In [224]:
plantParameters_merge_prices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35036 entries, 0 to 35035
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date_x                  35036 non-null  datetime64[ns]
 1   SettlementPoint         35036 non-null  object        
 2   Price_x                 35036 non-null  float64       
 3   Date1                   35036 non-null  datetime64[ns]
 4   Year                    35036 non-null  int64         
 5   index                   35036 non-null  int64         
 6   PlantName               35036 non-null  object        
 7   Year_x                  35036 non-null  int64         
 8   Month_x                 35036 non-null  int64         
 9   PowerPriceName          35036 non-null  object        
 10  FuelPriceName_x         35036 non-null  object        
 11  Capacity                35036 non-null  int64         
 12  HeatRate                35036 non-null  float6

## Task 12: Identify hours in which the power plant should be on

In [235]:
plantParameters_merge_prices['Generation'] = 0
plantParameters_merge_prices['Gen_ix'] = 0
ix_gen = plantParameters_merge_prices['Price_x'] > plantParameters_merge_prices['RunningCost']
plantParameters_merge_prices.loc[ix_gen, 'Generation'] = plantParameters_merge_prices.loc[ix_gen, 'Capacity']
plantParameters_merge_prices.loc[ix_gen, 'Gen_ix'] = 1

In [236]:
plantParameters_merge_prices

,Date_x,SettlementPoint,Price_x,Date1,Year,index,PlantName,Year_x,Month_x,PowerPriceName,...,FixedStartCost,FuelPriceName_y,Date_y,Price_y,Year_y,Month_y,RunningCost,Generation,RunningMargin,Gen_ix
0,2017-01-01 00:00:00,HB_HOUSTON,23.23,2017-01-01,2017,2313,Alpha,2017,1,HB_HOUSTON,...,10000,Henry Hub,2017-01-01,3.65,2017,1,37.500,0,-0.0,0
2,2017-01-01 01:00:00,HB_HOUSTON,21.82,2017-01-01,2017,2313,Alpha,2017,1,HB_HOUSTON,...,10000,Henry Hub,2017-01-01,3.65,2017,1,37.500,0,-0.0,0
4,2017-01-01 02:00:00,HB_HOUSTON,20.16,2017-01-01,2017,2313,Alpha,2017,1,HB_HOUSTON,...,10000,Henry Hub,2017-01-01,3.65,2017,1,37.500,0,-0.0,0
6,2017-01-01 03:00:00,HB_HOUSTON,20.00,2017-01-01,2017,2313,Alpha,2017,1,HB_HOUSTON,...,10000,Henry Hub,2017-01-01,3.65,2017,1,37.500,0,-0.0,0
8,2017-01-01 04:00:00,HB_HOUSTON,20.24,2017-01-01,2017,2313,Alpha,2017,1,HB_HOUSTON,...,10000,Henry Hub,2017-01-01,3.65,2017,1,37.500,0,-0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35027,2018-12-31 19:00:00,HB_SOUTH,28.66,2018-12-31,2018,173175,Bravo,2018,12,HB_SOUTH,...,50000,GDA_TETSTX,2018-12-31,3.08,2018,12,25.788,250,718.0,1
35029,2018-12-31 20:00:00,HB_SOUTH,26.57,2018-12-31,2018,173175,Bravo,2018,12,HB_SOUTH,...,50000,GDA_TETSTX,2018-12-31,3.08,2018,12,25.788,250,195.5,1
35031,2018-12-31 21:00:00,HB_SOUTH,24.09,2018-12-31,2018,173175,Bravo,2018,12,HB_SOUTH,...,50000,GDA_TETSTX,2018-12-31,3.08,2018,12,25.788,0,-0.0,0
35033,2018-12-31 22:00:00,HB_SOUTH,22.67,2018-12-31,2018,173175,Bravo,2018,12,HB_SOUTH,...,50000,GDA_TETSTX,2018-12-31,3.08,2018,12,25.788,0,-0.0,0


In [237]:
# RunningMargin = (PowerPrice – RunningCost) * Generation
plantParameters_merge_prices['RunningMargin'] = (plantParameters_merge_prices['Price_x']-plantParameters_merge_prices['RunningCost'])*plantParameters_merge_prices['Generation']

In [238]:
plantParameters_merge_prices

,Date_x,SettlementPoint,Price_x,Date1,Year,index,PlantName,Year_x,Month_x,PowerPriceName,...,FixedStartCost,FuelPriceName_y,Date_y,Price_y,Year_y,Month_y,RunningCost,Generation,RunningMargin,Gen_ix
0,2017-01-01 00:00:00,HB_HOUSTON,23.23,2017-01-01,2017,2313,Alpha,2017,1,HB_HOUSTON,...,10000,Henry Hub,2017-01-01,3.65,2017,1,37.500,0,-0.0,0
2,2017-01-01 01:00:00,HB_HOUSTON,21.82,2017-01-01,2017,2313,Alpha,2017,1,HB_HOUSTON,...,10000,Henry Hub,2017-01-01,3.65,2017,1,37.500,0,-0.0,0
4,2017-01-01 02:00:00,HB_HOUSTON,20.16,2017-01-01,2017,2313,Alpha,2017,1,HB_HOUSTON,...,10000,Henry Hub,2017-01-01,3.65,2017,1,37.500,0,-0.0,0
6,2017-01-01 03:00:00,HB_HOUSTON,20.00,2017-01-01,2017,2313,Alpha,2017,1,HB_HOUSTON,...,10000,Henry Hub,2017-01-01,3.65,2017,1,37.500,0,-0.0,0
8,2017-01-01 04:00:00,HB_HOUSTON,20.24,2017-01-01,2017,2313,Alpha,2017,1,HB_HOUSTON,...,10000,Henry Hub,2017-01-01,3.65,2017,1,37.500,0,-0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35027,2018-12-31 19:00:00,HB_SOUTH,28.66,2018-12-31,2018,173175,Bravo,2018,12,HB_SOUTH,...,50000,GDA_TETSTX,2018-12-31,3.08,2018,12,25.788,250,718.0,1
35029,2018-12-31 20:00:00,HB_SOUTH,26.57,2018-12-31,2018,173175,Bravo,2018,12,HB_SOUTH,...,50000,GDA_TETSTX,2018-12-31,3.08,2018,12,25.788,250,195.5,1
35031,2018-12-31 21:00:00,HB_SOUTH,24.09,2018-12-31,2018,173175,Bravo,2018,12,HB_SOUTH,...,50000,GDA_TETSTX,2018-12-31,3.08,2018,12,25.788,0,-0.0,0
35033,2018-12-31 22:00:00,HB_SOUTH,22.67,2018-12-31,2018,173175,Bravo,2018,12,HB_SOUTH,...,50000,GDA_TETSTX,2018-12-31,3.08,2018,12,25.788,0,-0.0,0


## Task 13: Account for start costs

In [239]:
plantParameters_merge_prices.sort_values(by=['PlantName', 'Date_x'], inplace=True)

In [240]:
plantParameters_merge_prices

,Date_x,SettlementPoint,Price_x,Date1,Year,index,PlantName,Year_x,Month_x,PowerPriceName,...,FixedStartCost,FuelPriceName_y,Date_y,Price_y,Year_y,Month_y,RunningCost,Generation,RunningMargin,Gen_ix
0,2017-01-01 00:00:00,HB_HOUSTON,23.23,2017-01-01,2017,2313,Alpha,2017,1,HB_HOUSTON,...,10000,Henry Hub,2017-01-01,3.65,2017,1,37.500,0,-0.0,0
2,2017-01-01 01:00:00,HB_HOUSTON,21.82,2017-01-01,2017,2313,Alpha,2017,1,HB_HOUSTON,...,10000,Henry Hub,2017-01-01,3.65,2017,1,37.500,0,-0.0,0
4,2017-01-01 02:00:00,HB_HOUSTON,20.16,2017-01-01,2017,2313,Alpha,2017,1,HB_HOUSTON,...,10000,Henry Hub,2017-01-01,3.65,2017,1,37.500,0,-0.0,0
6,2017-01-01 03:00:00,HB_HOUSTON,20.00,2017-01-01,2017,2313,Alpha,2017,1,HB_HOUSTON,...,10000,Henry Hub,2017-01-01,3.65,2017,1,37.500,0,-0.0,0
8,2017-01-01 04:00:00,HB_HOUSTON,20.24,2017-01-01,2017,2313,Alpha,2017,1,HB_HOUSTON,...,10000,Henry Hub,2017-01-01,3.65,2017,1,37.500,0,-0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35027,2018-12-31 19:00:00,HB_SOUTH,28.66,2018-12-31,2018,173175,Bravo,2018,12,HB_SOUTH,...,50000,GDA_TETSTX,2018-12-31,3.08,2018,12,25.788,250,718.0,1
35029,2018-12-31 20:00:00,HB_SOUTH,26.57,2018-12-31,2018,173175,Bravo,2018,12,HB_SOUTH,...,50000,GDA_TETSTX,2018-12-31,3.08,2018,12,25.788,250,195.5,1
35031,2018-12-31 21:00:00,HB_SOUTH,24.09,2018-12-31,2018,173175,Bravo,2018,12,HB_SOUTH,...,50000,GDA_TETSTX,2018-12-31,3.08,2018,12,25.788,0,-0.0,0
35033,2018-12-31 22:00:00,HB_SOUTH,22.67,2018-12-31,2018,173175,Bravo,2018,12,HB_SOUTH,...,50000,GDA_TETSTX,2018-12-31,3.08,2018,12,25.788,0,-0.0,0


In [258]:
condition = plantParameters_merge_prices.Gen_ix.eq(0)
plantParameters_merge_prices['Reset'] = condition.groupby(plantParameters_merge_prices.PlantName).cumsum()

In [260]:
plantParameters_merge_prices['Cumulative'] = plantParameters_merge_prices.RunningMargin.mask(condition, 0).groupby([plantParameters_merge_prices.PlantName, plantParameters_merge_prices.Reset]).cumsum()

In [270]:
ix_start = plantParameters_merge_prices['Cumulative']>plantParameters_merge_prices['FixedStartCost']
plantParameters_merge_prices['Generation2']=0

In [271]:
plantParameters_merge_prices.loc[ix_start, 'Generation2'] = plantParameters_merge_prices.loc[ix_start, 'Capacity']

In [272]:
plantParameters_merge_prices

,Date_x,SettlementPoint,Price_x,Date1,Year,index,PlantName,Year_x,Month_x,PowerPriceName,...,Price_y,Year_y,Month_y,RunningCost,Generation,RunningMargin,Gen_ix,Cumulative,Reset,Generation2
0,2017-01-01 00:00:00,HB_HOUSTON,23.23,2017-01-01,2017,2313,Alpha,2017,1,HB_HOUSTON,...,3.65,2017,1,37.500,0,-0.0,0,0.0,1,0
2,2017-01-01 01:00:00,HB_HOUSTON,21.82,2017-01-01,2017,2313,Alpha,2017,1,HB_HOUSTON,...,3.65,2017,1,37.500,0,-0.0,0,0.0,2,0
4,2017-01-01 02:00:00,HB_HOUSTON,20.16,2017-01-01,2017,2313,Alpha,2017,1,HB_HOUSTON,...,3.65,2017,1,37.500,0,-0.0,0,0.0,3,0
6,2017-01-01 03:00:00,HB_HOUSTON,20.00,2017-01-01,2017,2313,Alpha,2017,1,HB_HOUSTON,...,3.65,2017,1,37.500,0,-0.0,0,0.0,4,0
8,2017-01-01 04:00:00,HB_HOUSTON,20.24,2017-01-01,2017,2313,Alpha,2017,1,HB_HOUSTON,...,3.65,2017,1,37.500,0,-0.0,0,0.0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35027,2018-12-31 19:00:00,HB_SOUTH,28.66,2018-12-31,2018,173175,Bravo,2018,12,HB_SOUTH,...,3.08,2018,12,25.788,250,718.0,1,4206.5,10311,0
35029,2018-12-31 20:00:00,HB_SOUTH,26.57,2018-12-31,2018,173175,Bravo,2018,12,HB_SOUTH,...,3.08,2018,12,25.788,250,195.5,1,4402.0,10311,0
35031,2018-12-31 21:00:00,HB_SOUTH,24.09,2018-12-31,2018,173175,Bravo,2018,12,HB_SOUTH,...,3.08,2018,12,25.788,0,-0.0,0,0.0,10312,0
35033,2018-12-31 22:00:00,HB_SOUTH,22.67,2018-12-31,2018,173175,Bravo,2018,12,HB_SOUTH,...,3.08,2018,12,25.788,0,-0.0,0,0.0,10313,0


## Task 14: Write the results to file

In [275]:
plantParameters_merge_prices.to_csv('PowerPlantDispatch.csv')